# Dynamic risk budgeting between PSP & LHP

In [1]:
import pandas as pd
import numpy as np
import ashmodule as ash
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
n_scenarios = 5000
rates, zc_prices = ash.cir(10,n_scenarios=n_scenarios,b=0.03,r_0 = 0.03,sigma = 0.02)
prices_eq = ash.gbm(10, n_scenarios=n_scenarios,mu=0.07,sigma=0.15)

In [3]:
rets_eq = prices_eq.pct_change().dropna()
rets_zc = zc_prices.pct_change().dropna()
rets_7030b = ash.bt_mix(rets_eq, rets_zc,allocator = ash.fixedmix_allocator, w1 = 0.7)
pd.concat([ash.terminal_stats(rets_zc, name = "ZC", floor = 0.75),
           ash.terminal_stats(rets_eq, name = "Eq", floor = 0.75),
           ash.terminal_stats(rets_7030b, name = "70/30", floor = 0.75)],
         axis = 1).round(2)

,ZC,Eq,70/30
mean,1.34,1.98,1.76
std,0.00,0.97,0.59
p_breach,NaN,0.03,0.01
e_short,NaN,0.13,0.10
p_reach,NaN,NaN,NaN
e_surplus,NaN,NaN,NaN


In [5]:
rets_floor75 = ash.bt_mix(rets_eq, rets_zc, allocator=ash.floor_allocator, floor =0.75, zc_prices=zc_prices[1:])
pd.concat([ash.terminal_stats(rets_zc, name = "ZC", floor = 0.75),
           ash.terminal_stats(rets_eq, name = "Eq", floor = 0.75),
           ash.terminal_stats(rets_7030b, name = "70/30", floor = 0.75),
           ash.terminal_stats(rets_floor75, name ="Floor75%",floor=0.75)],
         axis = 1).round(2)

,ZC,Eq,70/30,Floor75%
mean,1.34,1.98,1.76,1.95
std,0.00,0.97,0.59,0.97
p_breach,NaN,0.03,0.01,NaN
e_short,NaN,0.13,0.10,NaN
p_reach,NaN,NaN,NaN,NaN
e_surplus,NaN,NaN,NaN,NaN


In [13]:
rets_floor75m1 = ash.bt_mix(rets_eq, rets_zc, allocator=ash.floor_allocator, floor =0.75, zc_prices=zc_prices[1:],m=1)
rets_floor75m5 = ash.bt_mix(rets_eq, rets_zc, allocator=ash.floor_allocator, floor =0.75, zc_prices=zc_prices[1:],m=5)
pd.concat([ash.terminal_stats(rets_zc, name = "ZC", floor = 0.75),
           ash.terminal_stats(rets_eq, name = "Eq", floor = 0.75),
           ash.terminal_stats(rets_7030b, name = "70/30", floor = 0.75),
           ash.terminal_stats(rets_floor75, name ="Floor75%",floor=0.75),
           ash.terminal_stats(rets_floor75m1, name ="Floor75%m1",floor=0.75),
           ash.terminal_stats(rets_floor75m5, name ="Floor75%m5",floor=0.75)],
         axis = 1).round(2)

,ZC,Eq,70/30,Floor75%,Floor75%m1,Floor75%m5
mean,1.34,1.98,1.76,1.95,1.62,1.96
std,0.00,0.97,0.59,0.97,0.43,0.97
p_breach,NaN,0.03,0.01,NaN,NaN,NaN
e_short,NaN,0.13,0.10,NaN,NaN,NaN
p_reach,NaN,NaN,NaN,NaN,NaN,NaN
e_surplus,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
pd.concat([ash.terminal_stats(rets_zc, name = "ZC", floor = 0.75),
           ash.terminal_stats(rets_eq, name = "Eq", floor = 0.75),
           ash.terminal_stats(rets_7030b, name = "70/30", floor = 0.75),
           ash.terminal_stats(rets_floor75, name ="Floor75%",floor=0.75),
           ash.terminal_stats(rets_floor75m1, name ="Floor75%m1",floor=0.75),
           ash.terminal_stats(rets_floor75m5, name ="Floor75%m5",floor=0.75)],
         axis = 1).round(5)

,ZC,Eq,70/30,Floor75%,Floor75%m1,Floor75%m5
mean,1.34334,1.97530,1.76113,1.95263,1.62436,1.96282
std,0.00000,0.96566,0.58989,0.96994,0.42616,0.97251
p_breach,NaN,0.03460,0.00840,NaN,NaN,NaN
e_short,NaN,0.12635,0.09649,NaN,NaN,NaN
p_reach,NaN,NaN,NaN,NaN,NaN,NaN
e_surplus,NaN,NaN,NaN,NaN,NaN,NaN


# Risk Budgeting with Drawdown Constraint

In [17]:
cashrate = 0.02
monthly_cashreturn = (1+cashrate)**(1/12) -1
rets_cash = pd.DataFrame(data = monthly_cashreturn, index = rets_eq.index, columns=rets_eq.columns)
rets_maxdd25 = ash.bt_mix(rets_eq, rets_cash,allocator=ash.drawdown_allocator,maxdd=0.25)
tv_maxdd25 = ash.terminal_values(rets_maxdd25)
pd.concat([ash.terminal_stats(rets_zc, name = "ZC", floor = 0.75),
           ash.terminal_stats(rets_eq, name = "Eq", floor = 0.75),
           ash.terminal_stats(rets_7030b, name = "70/30", floor = 0.75),
           ash.terminal_stats(rets_floor75, name ="Floor75%",floor=0.75),
           ash.terminal_stats(rets_floor75m1, name ="Floor75%m1",floor=0.75),
           ash.terminal_stats(rets_floor75m5, name ="Floor75%m5",floor=0.75),
           ash.terminal_stats(rets_maxdd25, name ="MaxDD25",floor=0.75)
          ],
         axis = 1).round(5)

,ZC,Eq,70/30,Floor75%,Floor75%m1,Floor75%m5,MaxDD25
mean,1.34334,1.97530,1.76113,1.95263,1.62436,1.96282,1.63424
std,0.00000,0.96566,0.58989,0.96994,0.42616,0.97251,0.53411
p_breach,NaN,0.03460,0.00840,NaN,NaN,NaN,NaN
e_short,NaN,0.12635,0.09649,NaN,NaN,NaN,NaN
p_reach,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e_surplus,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
ash.summary_stats(rets_maxdd25)["Max Drawdown"].head()

0   -0.151729
1   -0.185550
2   -0.219393
3   -0.174793
4   -0.143515
Name: Max Drawdown, dtype: float64

In [21]:
ash.summary_stats(rets_maxdd25)["Max Drawdown"].min()

-0.23444253646059715